<a href="https://colab.research.google.com/github/tobiasgobel/PDFviewer/blob/main/VQE_Clifford_grid_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
from sympy.series import series
from scipy.optimize import minimize, NonlinearConstraint
import time
import itertools
from sympy import symbols, Matrix, SparseMatrix, cos, sin, expand, lambdify, O
from sympy.utilities.iterables import multiset_permutations

from functools import *
from operator import *
import scipy
import sympy
import numpy as np
import random
import math
from numba import jit

In [181]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r  took: %2.4f sec' %(f.__name__, te-ts))
        return result
    return wrap

Functions

We denote the ansatz as following:
ansatz = ["X0Y1","X1Y2","X0X1Y2"]

In [207]:
class Operators:
  def __init__(self, ansatz, N):
    self.ansatz = ansatz
    self.N = N
  def split(self):
      paulis = []
      positions = []
      for i in range(len(self.ansatz)):
          pauli_lst = []
          pos_lst = []
          #creating lists of operators and corresponding positions
          prev_int = False
          for k in self.ansatz[i]:
              if k.isdigit():
                  if not prev_int:
                      pos_lst.append(k)
                  else:
                      pos_lst[-1] += k
                  prev_int = True
              else:
                  pauli_lst.append(k)
                  prev_int = False
          paulis.append(pauli_lst)
          positions.append(pos_lst)
      return positions, paulis
  @jit(nopython = True)
  def matrix_repr(self):
    positions, paulis = self.split()
    matrices = []
    for i in range(len(self.ansatz)):
      Kron = 1
      counter = 0
      for j in range(self.N):
        if counter == len(positions[i]):
          for _ in range(self.N-j):
            Kron = np.kron(Kron, Pauli["I"])
          break
        elif j == int(positions[i][counter]):
          Kron = np.kron(Kron, Pauli[paulis[i][counter]])
          counter+=1
        else:
          Kron = np.kron(Kron, Pauli["I"])
      matrices += [Kron]
    return matrices

def pauli_on_pauli(p1,p2):
    
    if p1 == 'X' and p2 == 'Y':
        return 1j, 'Z'
    elif p1 == 'X' and p2 == 'X':
        return 1, 'I'
    elif p1 == 'Y' and p2 == 'Y':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'Z':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'X':
        return 1j, 'Y'
    elif p1 == 'Z' and p2 == 'Y':
        return -1j, 'X'
    elif p1 == 'I':
        return 1, p2
    elif p2 == 'I':
        return 1, p1
    else:
        a, p = pauli_on_pauli(p2,p1)
        return -1*a, p

def single_pauli_action(pauli, spin):
    
    if pauli=='X':
        return((spin+1)%2, 1)
    elif pauli=='Y':
        return((spin+1)%2, 1j*(-1)**spin)
    elif pauli=='Z':
        return(spin, (-1)**spin)
    elif pauli=='I':
        return(spin, 1)
    else:
        print('wrong pauli!')
        return(None)

def findCombinationsUtil(li, arr, index, num, reducedNum):
    z = []
    if (reducedNum < 0): 
        return; 
    if (reducedNum == 0): 
  
        for i in range(index): 
            z = z + [arr[i]]
        li.append(z) 
        return;

    prev = 1 if (index == 0) else arr[index - 1]; 
  
    for k in range(prev, num + 1): 
          

        arr[index] = k; 
  
        findCombinationsUtil(li,arr, index + 1, num,  
                                 reducedNum - k); 
    return li

def k_all(N, generators, order): 
      
    # array to store the combinations 
    # It can contain max n elements
    out = []
    k_length = len(generators)
    for k in range(1, order+1):
        arr = [0] * k;
        output = []
        a =  findCombinationsUtil([], arr, 0, k, k);
        for i in a:
            if len(i)<= k_length:
                i = i.extend((k_length-len(i))*[0])
        for j in a:
            if len(j) == k_length:
#                 if k_vector(N, interactions,j).state()[1] != N*[0]:
                output = output + list(multiset_permutations(j))
        out =  out+ output
    return [tuple(p) for p in [[0]*k_length] + out]

def power_product(x,y):
    out = 1
    for i in range(len(x)):
         out*= x[i]**y[i]
    return out



In [192]:
class pauli:
  def __init__(self,string, N, factor = 1):
    self.string = string
    self.factor = factor
    self.N = N
    self.starting_state = np.array([0]*self.N)

  def __str__(self):
    return self.string+".   factor: "+str(self.factor)
    
  #define multiplying by a constant (on left hand side)
  def __rmul__(self, c):
    self.factor *= c
    return self

  #define the power of a pauli string
  def __pow__(self, c): 
    C = pauli("I0",self.N)
    for i in range(abs(c)):
      C = C*self
    return C

  #define multiplying two pauli strings
  def __mul__(self, x):
    pos1, pauli1 = self.split()
    pos2, pauli2 = x.split()
    factor = self.factor*x.factor
    string = ""
    counter1 =0
    counter2 =0

    for j in range(self.N):
      end1 = counter1 == len(pos1)
      end2 = counter2 == len(pos2)

      if not end1 and not end2:
        if int(pos1[counter1]) == j and int(pos2[counter2]) == j:
          a, p= pauli_on_pauli(pauli1[counter1],pauli2[counter2])
          factor *= a
          string+= p+str(j)
          counter1+=1
          counter2+=1
        elif int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
        elif int(pos2[counter2]) == j:
          string+=pauli2[counter2]+str(j)
          counter2+=1
      elif not end1:
        if int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
      elif not end2:
          if int(pos2[counter2]) == j:
            string+=pauli2[counter2]+str(j)
            counter2+=1
      else:
        pass
      
    return pauli(string, self.N, factor)

  #calculate resulting state of paulistring when acted upon initial_state  
  def state(self, initial_state = 0):
    pos, pauli = self.split()
    init_state = self.starting_state + initial_state
    a = self.factor
    for j in range(len(pos)):
      Pauli = pauli[j]
      spin = init_state[int(pos[j])]
      new_spin, factor = single_pauli_action(Pauli,spin)
      init_state[int(pos[j])] = new_spin
      a *= factor
    return a, tuple(init_state)

    
#creating lists of operators and corresponding positions
  def split(self):
    pauli_lst = []
    pos_lst = []
    prev_int = False
    for k in self.string:
        if k.isdigit():
            if not prev_int:
                pos_lst.append(k)
            else:
                pos_lst[-1] += k
            prev_int = True
        else:
            pauli_lst.append(k)
            prev_int = False
    return pos_lst, pauli_lst
  

  

In [184]:
#gives result of transformation exp(-i*T1)*T2*exp(i*T2)
def Clifford_map(T1, T2, reversed_arguments = True):
  T1T2 = T1*T2
  T2T1 = T2*T1

  if T1T2.factor == T2T1.factor:
    if reversed_arguments:
      return T1
    else:
      return T2
  elif T1T2.factor == -T2T1.factor:
    if reversed_arguments:
      return -1j*T2T1
    else:
      return -1j*T1T2
  else:
    return "something wrong here"


#returns list of pauli objects that are the result 
#of pulling all clifford gates to the left
def pull_cliffords_through(ansatz, K, N):
  T_K = [ansatz[0]]
  
  for j in range(1, len(ansatz)):
    T = ansatz[j]
    for i in range(j-1,-1,-1):
      for _ in range(K[i]):
        T = Clifford_map(T,ansatz[i])
    T_K += [T] 
  return T_K





In [212]:
#creates a dictionary of states that contain contributions

def s_dict(N, ansatz, K, order):

  lst = list(map(list, itertools.product([0, 1], repeat=N))) #all possible bitstrings
  s_dict = {} #keys: possible bitstrings, values dictionary with orders
  T_K = pull_cliffords_through(ansatz, K, N)
  K_all = k_all(N, ansatz, order)

  for i in K_all: #loop through all 

    #calculate state that is produced by T_i
    pauli_string = power_product(T_K[::-1], i[::-1])
    factor, state = pauli_string.state()

    #calculate magnitude of term
    factorial = np.prod(np.array([math.factorial(j) for j in i]))
    term = (1j)**sum(i)*factor/factorial

    #check whether binary string is in dictionary, otherwise add
    if state not in s_dict:
      s_dict[state] = {}
    if sum(i) not in s_dict[state]:
      s_dict[state][sum(i)] = {}

    #add to right place in dictionary
    s_dict[state][sum(i)][i]= term

  return s_dict


def G_k(N, H, ansatz, K):
  g_k = []
  for P in H:
    #Initialize list of Clifford gates with respective power of K.
    G_K = [ansatz[i]**K[i] for i in range(len(K))]

    #Apply nested Clifford Map to obtain G^-K P_a G^K
    paulistring = reduce(Clifford_map, [P]+G_K[::-1])

    g_k += [paulistring]
  return g_k


def dict_add(d, thetas):
  a = 0
  
  for i in d:
    a+= power_product(thetas, i)*d[i]
  np.sum(thetas**d)
  return a


In [216]:
@timing
def energy(thetas, H,ansatz, s_dict, K,G_K, order):
  E = 0
  s_dict1 = s_dict
  for paulistring in G_K: #loop through terms in Hamiltonian
    E_a = 0

    #loop over basis states
    for s in s_dict1:
      E_a_s = 0
    
      #Calculate G^-K P_a G^K |s>
      a, state = paulistring.state(s)

      #Define contributions of |s> and |s'>
      psi_s1 = s_dict1[s]
      psi_s2 = s_dict1[state]
      #Double for loop to take the right orders in perturbation theory
      for o1 in psi_s1:
        for o2 in range(order - o1 +1):
          if o2 in psi_s2:
            A = dict_add(psi_s1[o1],thetas)
            B = dict_add(psi_s2[o2],thetas)
            E_a_s += A*np.conj(B)

      E_a_s *= a
      E_a += E_a_s
    
    E += E_a
  return np.real(E)


In [187]:
class K_hopping:
  def __init__(self,N, ansatz, H, order, K_init):
    self.N = N
    self.ansatz = ansatz
    self.H = H
    self.order = order
    self.K = K_init
    self.path = [K_init.copy()]
    self.len = len(ansatz)

  def evaluate(self):
    #init thetas
    initial_guess = [0]*self.len

    #initializing s-dictionary and clifford gates
    s = s_dict(self.N, self.ansatz, self.K, self.order)
    G_K = G_k(self.N, self.H, self.ansatz, self.K)

    #scipy minimize around self.K
    result = scipy.optimize.minimize(energy, initial_guess,jac = False, args = (self.H,self.ansatz,s,self.K,G_K,self.order))

    return result

  def step(self, result):
    #determine index largest angle
    thetas = result.x
    index = np.argmax(np.abs(thetas))
    print(result.x)
    if thetas[index] >= np.pi/8:
      self.K[index] += 1
    elif thetas[index] <= -np.pi/8:
      self.K[index] += 3
    else:
      return False
    self.path += [self.K.copy()]
    print(self.K)
    return True

  @timing
  def hopping(self,iters=10):
    next = True
    for _ in range(iters):
      result = self.evaluate()
      next = self.step(result)
      if not next:
        return result
    return result





In [197]:
N = 2
ansatz = [pauli("X0Y1",N)]#,pauli("X1Y2",N),pauli("X2Y3",N)]

Z_h = -.01
X_h = -1
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h)]#,pauli("Z2",N,Z_h),pauli("Z3",N,Z_h),pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h)]
H+= [pauli("X0X1",N,X_h)]
K = [0]
order = 3

# instance = K_hopping(N,ansatz, H, order, K)
# print(instance.hopping(), instance.path)

In [189]:
print(1.23664/np.pi)
np.pi**2/8

0.80873673/np.pi

0.3936347376503229


0.25742889647895106

In [217]:
N = 4
ansatz = [pauli("X0Y1",N),pauli("X1Y2",N),pauli("X2Y3",N)]

Z_h = -.1
X_h = -1
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h),pauli("Z3",N,Z_h),pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h)]
# H+= [pauli("X0X1",N,X_h)]
K = [0,0,0]
order = 9
thetas=[0,0,0]
s = s_dict(N, ansatz, K, order)
G_K = G_k(N, H, ansatz, K)

print(s)
# print([str(i) for i in G_K])

print(energy(thetas, H, ansatz, s, K, G_K, order))

{(0, 0, 0, 0): {0: {(0, 0, 0): (1+0j)}, 2: {(0, 0, 2): (-0.5+0j), (0, 2, 0): (-0.5+0j), (2, 0, 0): (-0.5+0j)}, 4: {(0, 2, 2): (0.25+0j), (2, 0, 2): (0.25+0j), (2, 2, 0): (0.25+0j), (0, 0, 4): (0.041666666666666664+0j), (0, 4, 0): (0.041666666666666664+0j), (4, 0, 0): (0.041666666666666664+0j)}, 6: {(2, 2, 2): (-0.125+0j), (0, 2, 4): (-0.020833333333333332+0j), (0, 4, 2): (-0.020833333333333332+0j), (2, 0, 4): (-0.020833333333333332+0j), (2, 4, 0): (-0.020833333333333332+0j), (4, 0, 2): (-0.020833333333333332+0j), (4, 2, 0): (-0.020833333333333332+0j), (0, 0, 6): (-0.001388888888888889+0j), (0, 6, 0): (-0.001388888888888889+0j), (6, 0, 0): (-0.001388888888888889+0j)}, 8: {(2, 2, 4): (0.010416666666666666+0j), (2, 4, 2): (0.010416666666666666+0j), (4, 2, 2): (0.010416666666666666+0j), (0, 2, 6): (0.0006944444444444445+0j), (0, 6, 2): (0.0006944444444444445+0j), (2, 0, 6): (0.0006944444444444445+0j), (2, 6, 0): (0.0006944444444444445+0j), (6, 0, 2): (0.0006944444444444445+0j), (6, 2, 0): 

In [218]:
initial_guess = [0]*len(ansatz)
result = scipy.optimize.minimize(energy, initial_guess,jac = False, args = (H,ansatz,s,K,G_K,order))

func:'energy'  took: 0.0289 sec
func:'energy'  took: 0.0222 sec
func:'energy'  took: 0.0206 sec
func:'energy'  took: 0.0218 sec
func:'energy'  took: 0.0207 sec
func:'energy'  took: 0.0212 sec
func:'energy'  took: 0.0216 sec
func:'energy'  took: 0.0220 sec
func:'energy'  took: 0.0210 sec
func:'energy'  took: 0.0216 sec
func:'energy'  took: 0.0278 sec
func:'energy'  took: 0.0226 sec
func:'energy'  took: 0.0204 sec
func:'energy'  took: 0.0230 sec
func:'energy'  took: 0.0220 sec
func:'energy'  took: 0.0223 sec
func:'energy'  took: 0.0217 sec
func:'energy'  took: 0.0217 sec
func:'energy'  took: 0.0233 sec
func:'energy'  took: 0.0202 sec
func:'energy'  took: 0.0241 sec
func:'energy'  took: 0.0231 sec
func:'energy'  took: 0.0232 sec
func:'energy'  took: 0.0224 sec
func:'energy'  took: 0.0208 sec
func:'energy'  took: 0.0217 sec
func:'energy'  took: 0.0218 sec
func:'energy'  took: 0.0249 sec
func:'energy'  took: 0.0207 sec
func:'energy'  took: 0.0219 sec
func:'energy'  took: 0.0256 sec
func:'en

In [154]:
1/6

0.16666666666666666

In [124]:
print(result)

      fun: -1.500455998397633
 hess_inv: array([[ 0.12452819, -0.00018407,  0.00102221],
       [-0.00018407,  0.06356769, -0.00111826],
       [ 0.00102221, -0.00111826,  0.06275287]])
      jac: array([-4.47034836e-08, -1.78813934e-07, -1.19209290e-07])
  message: 'Optimization terminated successfully.'
     nfev: 132
      nit: 13
     njev: 33
   status: 0
  success: True
        x: array([-1.21980401,  0.00618522, -1.22226686])


In [193]:
np.pi/4

0.7853981633974483

In [153]:
print(sum(dict['aa']))

TypeError: ignored